In [1]:
import json

In [2]:
from os.path import join, dirname
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource

In [3]:
import threading
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pandas as pd

In [4]:
authenticator = IAMAuthenticator("TV_4eGc6BAyMwhHXYXQM32545uTaNB4EquRKnN9BQX4w")
service = SpeechToTextV1(authenticator=authenticator)
service.set_service_url('https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/24ac02e4-9660-4ee2-905c-95b2aa1b6b66')

In [18]:
originalWavPath = r'F:/Boolean-Lab/speech-expresssion-analysis/demo-implementation/__file__/testing-audio.wav'

In [19]:
models = service.list_models().get_result()
#print(json.dumps(models, indent=2))

model = service.get_model('en-US_BroadbandModel').get_result()
#print(json.dumps(model, indent=2))

with open(originalWavPath,'rb') as audio_file:
#    print(json.dumps(
    output = service.recognize(
    audio=audio_file,
    speaker_labels=True,
    content_type='audio/wav',
    #timestamps=True,
    #word_confidence=True,
    model='en-US_NarrowbandModel').get_result(),
    indent=2
df_1 = pd.DataFrame([i for elts in output for alts in elts['results'] for i in alts['alternatives']])
df_2 = pd.DataFrame([i for elts in output for i in elts['speaker_labels']])

In [186]:
df_2

,from,to,speaker,confidence,final
0,1.35,1.50,0,0.27,False
1,1.50,1.88,0,0.27,False
2,1.88,1.99,0,0.27,False
3,1.99,2.12,0,0.27,False
4,2.12,2.35,0,0.27,False
...,...,...,...,...,...
1036,357.81,358.02,2,0.63,False
1037,358.02,358.17,2,0.63,False
1038,358.17,358.53,2,0.63,False
1039,358.56,358.68,0,0.41,False


In [177]:
df_2['speaker'].unique()

array([0, 2], dtype=int64)

In [178]:
from pydub import AudioSegment

In [179]:
import scipy.io.wavfile as wavfile

In [180]:
newWavPathA = r'F:/Boolean-Lab/speech-expresssion-analysis/demo-implementation/__file__/trimA.wav'
newWavPathB = r'F:/Boolean-Lab/speech-expresssion-analysis/demo-implementation/__file__/trimB.wav'
wav_A  = AudioSegment.from_wav("welcome.wav")
wav_B = AudioSegment.from_wav("welcome.wav")
segments_A = [wav_A]
segments_B = [wav_B]
combined_A = AudioSegment.empty()
combined_B = AudioSegment.empty()
for a in range (len(df_2)):
    if df_2['speaker'][a] == 0:
        sampleRate, waveData = wavfile.read( originalWavPath )
        startSample = int( df_2['from'][a] * sampleRate )
        endSample = int( df_2['to'][a] * sampleRate )
        Wav = waveData[startSample:endSample]
        wavfile.write(f'diarization_cluster_A.wav', sampleRate, Wav)
        wav_a = AudioSegment.from_wav('diarization_cluster_A.wav')
        segments_A.append(wav_a)
    elif df_2['speaker'][a] == 2:
        sampleRate, waveData = wavfile.read( originalWavPath )
        startSample = int( df_2['from'][a] * sampleRate )
        endSample = int(  df_2['to'][a] * sampleRate )
        Wav = waveData[startSample:endSample]
        wavfile.write(f'diarization_cluster_B.wav', sampleRate, Wav)
        wav_b = AudioSegment.from_wav('diarization_cluster_B.wav')
        segments_B.append(wav_b)
    #print(label)
    
for fname_a in segments_A:
    combined_A += fname_a
for fname_b in segments_B:
    combined_B += fname_b
combined_A.export(newWavPathA, format="wav")
combined_B.export(newWavPathB, format="wav")
# wavfile.write( newWavPathA, sampleRate, wav_A)        
# wavfile.write( newWavPathB, sampleRate, wav_B) 

<_io.BufferedRandom name='F:/Boolean-Lab/speech-expresssion-analysis/demo-implementation/__file__/trimB.wav'>